# Class Project
## GenePattern Single Cell Analyses Workshop 

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Log in to GenePettern with your credentials.
</div>

In [17]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

<div class="alert alert-danger">
Clean up this section
</div>

In the Human Cell Atlas we can navigate thi this page:

from https://data.humancellatlas.org/explore/projects/cddab57b-6868-4be4-806f-395ed9dd635a/m/expression-matrices

Which can be accessed from: https://data.humancellatlas.org/explore/projects?filter=%5B%7B%22facetName%22%3A%22genusSpecies%22%2C%22terms%22%3A%5B%22Homo+sapiens%22%5D%7D%2C%7B%22facetName%22%3A%22organ%22%2C%22terms%22%3A%5B%22pancreas%22%5D%7D%5D

There, we will copy the link to the file which contains the data in the MTX format:
https://data.humancellatlas.org/project-assets/project-matrices/cddab57b-6868-4be4-806f-395ed9dd635a.homo_sapiens.mtx.zip

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
Paste the link to the `mtx.zip` file in the `url*` parameter below
</div>

In [71]:
import os 
import shutil
import urllib.request
import subprocess
import rpy2
%load_ext nbtools.r_support

@genepattern.build_ui(description="Setup the R and Python environments for the rest of this notebook. Downloads the example dataset to the notebook server.", 
                      parameters={"url":{"name":"url","default":"https://data.humancellatlas.org/project-assets/project-matrices/cddab57b-6868-4be4-806f-395ed9dd635a.homo_sapiens.mtx.zip"},
                                  "output_var": {"name":"folder","default":"folder","hide":"true"}
                                 })
def download_MTX_zip(url):
    %load_ext rpy2.ipython
    print("Retrieving input data...") 
    base_name = 'temp_data.mtx.zip'
    os.makedirs('data/input_data/unzipped_data', exist_ok=True)
    urllib.request.urlretrieve(url, f'data/input_data/{base_name}')
    subprocess.run(["unzip", f"data/input_data/"+f"{base_name}","-d",f"data/input_data/unzipped_data/"])
    folder = os.listdir("data/input_data/unzipped_data/")[0]
    folder = os.path.join("data/input_data/unzipped_data",folder)
    
    # cache the last downloaded copy if its a repeat (just in case)
    unzipPath = "data/input_data/unzipped_data/downloaded_MTX_folder"
    if os.path.exists(unzipPath):
        if os.path.exists(unzipPath + "_PREVIOUS"):
            shutil.rmtree(unzipPath + "_PREVIOUS")
        print("Saving old copy of the data as "+unzipPath + "_PREVIOUS")
        os.rename(unzipPath, unzipPath + "_PREVIOUS");  
        
    os.rename(folder,"data/input_data/unzipped_data/downloaded_MTX_folder")
    folder = 'data/input_data/unzipped_data/downloaded_MTX_folder'
    print(f'Data unzipped to: {folder}')
    print("Done.")
    return folder

The nbtools.r_support extension is already loaded. To reload it, use:
  %reload_ext nbtools.r_support


UIBuilder(description='Setup the R and Python environments for the rest of this notebook. Downloads the exampl…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
Input here the directory where your data resides. If you used default parameters, it should be in `data/input_data/unzipped_data/downloaded_MTX_folder`
</div>

In [70]:
%%r_build_ui { "name": "Setup Seurat Objects", "parameters": { "data_dir":{"name":"data_dir",},"output_var": { "hide": "True" } } }

setupR <- function(data_dir){
    
    write("Loading libraries...", stdout())
    suppressMessages(library(Seurat))
    suppressMessages(library(scater))
    # Load the dataset
    write(c("Reading data from",data_dir), stdout())
    suppressMessages(pbmc.data <- Read10X(data.dir = data_dir))
    
    # Initialize the Seurat object with the raw (non-normalized data).
    write("Loadig data into Seurat...", stdout())
    pbmc <- CreateSeuratObject(counts = pbmc.data, project = "pbmc3k", min.cells = 3, min.features = 200)
    write("Done with this step.", stdout())
    return(pbmc)
}
suppressMessages(pbmc <- setupR(data_dir))

UIBuilder(function_import='nbtools._r_wrappers["9C75C8590599CCF9A0352DB8AB5C2F39"]', name='Setup Seurat Object…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
Click `Run` to add mitochondiral QC metrics.
</div>

In [69]:
%%r_build_ui { "name": "Add Mitochondrial QC Metrics", "parameters": { "column_name": { "type": "string", "default":"percent.mt" },"pattern": { "type": "string", "default":"MT-" }, "output_var": { "hide": "True" } } }

set_mito_qc <- function(colName, pat) {
    write("Calculating the frequency of mitochondrial genes...", stdout())
    pattern <- paste("^", trimws(pat, which = "both"), sep="")
    
    # The [[ operator can add columns to object metadata. This is a great place to stash QC stats
    pbmc[[colName]] <- PercentageFeatureSet(pbmc, pattern = pattern)
    write("Done!", stdout())
    return(pbmc)
}


suppressMessages(pbmc <- set_mito_qc(column_name, pattern))

UIBuilder(function_import='nbtools._r_wrappers["5394781EB1FB7234C0D376B63D60B74D"]', name='Add Mitochondrial Q…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>

Plot the three features (nFeature_RNA, nCount_RNA, and percent.mt) to decide which filters to use in the next step.
</div>

In [68]:
%%r_build_ui -w 800 { "width": 10, "height": 300, "name": "Triple Violin Plot", "parameters": { "first_feature": { "type": "string", "default":"nFeature_RNA" }, "second_feature":{ "type": "string", "default":"nCount_RNA"}, "third_feature": { "type": "string", "default":"percent.mt" }, "output_var":{"hide":"True"} } }
# Visualize QC metrics as a violin plot
#VlnPlot(pbmc, features = c(first_feature, second_feature, third_feature), ncol = 3)
tripleViolin <- function(first, second, third){
     
    feats <- c(first, second, third)
    plot(VlnPlot(pbmc, features = feats, ncol = 3, combine=TRUE), fig.height=5, fig.width=15)
    return("")
}

tripleViolin(first_feature, second_feature, third_feature)

UIBuilder(function_import='nbtools._r_wrappers["2CE493410E1D9F5C44B7299CCA1B94F9"]', name='Triple Violin Plot'…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>

Based on the three violin plots above, input which filters to apply to the data.
</div>

In [67]:
%%r_build_ui { "name": "Subset Data", "parameters": { "min_n_features": { "type": "number", "default":"200" },"max_n_features": { "type": "number", "default":"9000" },"max_percent_mitochondrial": { "type": "number", "default":"50" }, "output_var": { "hide": "True" } } }

my_subset <- function(min_n_features, max_n_features, max_percent_mitochondrial){
#     print(pbmc)
    pbmc <- subset(pbmc, subset = nFeature_RNA > min_n_features & nFeature_RNA < max_n_features & percent.mt < max_percent_mitochondrial)
#     print(pbmc)
    write('filtering done!', stdout())
    return(pbmc)
}

pbmc <- my_subset(min_n_features, max_n_features, max_percent_mitochonrial)

UIBuilder(function_import='nbtools._r_wrappers["BEAF6867F4DD88D1D90E645F090AD1EB"]', name='Subset Data', origi…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Click run to apply a log-normalization and scaling of the data.
</div>

In [66]:

%%r_build_ui { "name": "Normalize", "parameters": { "method": { "type": "string", "default":"LogNormalize" },"scale_factor": { "type": "number", "default":"10000" }, "output_var": { "hide": "True" } } }

norm_pbmc <- function(meth, scale){
    write("Normalizing data...", stdout())
    invisible(pbmc <- NormalizeData(pbmc, normalization.method = meth, scale.factor = scale, verbose = F))
    write('Normalization done!', stdout())
    return(pbmc)
}

pbmc <- norm_pbmc(method, scale_factor)

UIBuilder(function_import='nbtools._r_wrappers["17C521EFD8671C919BEAB09B1E9844CA"]', name='Normalize', origin=…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Click run to show genes which are highly variable in this experiment. Take note of these genes as they may be helpful in downstream analyses.
</div>

In [65]:
%%r_build_ui { "name": "Feature Selection", "parameters": { "method": { "type": "string", "default":"vst","hide":"True" },"num_features": { "type": "number", "default":"2000" }, "num_to_label":{"type": "number", "default": "10", "description": "label the top N features in the plot."}, "output_var": { "hide": "True" } } }
#%%R -w 800 -h 450

feat_sel_plot <- function(meth, nFeat, nLabel){
    write("Identifying variable features...", stdout())
    invisible(capture.output(pbmc <- FindVariableFeatures(pbmc, selection.method = meth, nfeatures = nFeat, 
                                                         verbose=F)))
    write("Done!", stdout())

    # Identify the 10 most highly variable genes
    top10 <- head(VariableFeatures(pbmc), nLabel)

    # plot variable features with and without labels
    invisible(capture.output(plot1 <- VariableFeaturePlot(pbmc)))
    invisible(capture.output(plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)))
    print(plot2)
    #plot(CombinePlots(plots = list(plot1, plot2)))
    return(pbmc)
}

pbmc <- feat_sel_plot(method, num_features, num_to_label)

UIBuilder(function_import='nbtools._r_wrappers["F39D5C14DE7405F2A246EDCD0A7343A5"]', name='Feature Selection',…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Click run to compute PCA.
</div>

In [64]:
%%r_build_ui {"name": "Compute PCA", "parameters": {"output_var":{"hide": "True"}}}
myscale <- function(pbmc){
    write("Scaling data...", stdout())
    all.genes <- rownames(pbmc)
    invisible(capture.output(pbmc <- ScaleData(pbmc, features = all.genes, verbose = F)))
    write('Done scaling data!', stdout())
    
    feats <- VariableFeatures(object = pbmc, verbose = F)
    pbmc <- RunPCA(pbmc, features = feats, nfeatures.print=5, verbose=F)
    write('Done computing PCA. Elbow plot below.', stdout())
    
    plot(ElbowPlot(pbmc))
    
    return(pbmc)
}
pbmc <- myscale(pbmc)

UIBuilder(function_import='nbtools._r_wrappers["B4FD6D5F6DDD139951633655E86DAFD1"]', name='Compute PCA', origi…

In [63]:
%%r_build_ui {"name":"Save preprocessed dataset", "parameters": {  "file_name": {"default":"Seurat_preprocessed.rds"}, "output_var": {"hide": "True"} }}
save_it <- function(fileName){
    write('Saving file to the notebook workspace. This may take a while (this file may be aboyt 1 GB in size)...', stdout())
    saveRDS(pbmc, file = fileName)
    print("Saved file!")
    return(pbmc)
}
save_it(file_name)

UIBuilder(function_import='nbtools._r_wrappers["5FAEA968D1109686A4AC3270ED1C93C0"]', name='Save preprocessed d…

In [62]:
@nbtools.build_ui(name="Upload file to GenePattern Server", parameters={
    "file": {
        "name": "File to upload:",
        "default":"Seurat_preprocessed.rds"
    },
    "output_var": {
    "name": "results",
    "description": "",
    "default": "quantification_source",
    "hide": True
    }
})
def load_file(file):
    import genepattern
    uio = nbtools.UIOutput()
    display(uio)
    size = os.path.getsize(file)
    print(f'This file size is {round(size/1e6)} MB, it may take a while to upload.')
    uio.status = "Uploading..."
    uploaded_file = genepattern.session.get(0).upload_file(file_name=os.path.basename(file),file_path=file)
    uio.status = "Uploaded!"
    display(nbtools.UIOutput(files=[uploaded_file.get_url()]))
    return()

UIBuilder(function_import='nbtools.tool(id="Upload file to GenePattern Server", origin="Notebook").function_or…

### Hide this

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
If the above steps are taking too long (save+upload), feel free to use this.
</div>

In [2]:
display(nbtools.UIOutput(text='If the above steps are taking too long (save+upload), feel free to use this.',
                         files=['https://datasets.genepattern.org/data/module_support_files/SeuratClustering/Seurat_preprocessed_prebaked.rds']))

UIOutput(files=['https://datasets.genepattern.org/data/module_support_files/SeuratClustering/Seurat_preprocess…

## Seurat Clustering

In [18]:
seuratclustering_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00408')
seuratclustering_job_spec = seuratclustering_task.make_job_spec()
seuratclustering_job_spec.set_parameter("input.seurat.rds.file", "https://cloud.genepattern.org/gp/users/ted/tmp/run662634363058379919.tmp/Seurat_preprocessed.rds")
seuratclustering_job_spec.set_parameter("output.filename", "<input.seurat.rds.file_basename>.clustered")
seuratclustering_job_spec.set_parameter("maximum_dimension", "10")
seuratclustering_job_spec.set_parameter("resolution", "0.5")
seuratclustering_job_spec.set_parameter("reduction", "umap")
seuratclustering_job_spec.set_parameter("job.memory", "2 Gb")
seuratclustering_job_spec.set_parameter("job.queue", "gp-cloud-default")
seuratclustering_job_spec.set_parameter("job.cpuCount", "1")
seuratclustering_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(seuratclustering_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00408')

# Visualize Seurat results

## Download files from GPServer

#### pre-baked

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
If the above is not working, run the cell below to access the pre-ran results.
</div>

In [61]:
job199267 = gp.GPJob(genepattern.session.get(0), 199267)
genepattern.display(job199267)

GPJobWidget(job_number=199267)

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Download the CSV file.
</div>

In [14]:
import os

DownloadJobResultFile_description = "Download file from a GenePattern module job result."
DownloadJobResultFile_parameters = {"file": {"type": "file", "kinds": ["csv", "rds"]}, "output_var": {"hide": True}}   
        
def DownloadJobResultFile(file):
    # extract job number and file name from url
    job_num = file.split("/")[-2]
    remote_file_name = file.split("/")[-1]
    
    # get the job based on the job number passed as the second argument
    job = gp.GPJob(genepattern.get_session(0), job_num)

    # fetch a specific file from the job
    remote_file = job.get_file(remote_file_name)
    
    uio = nbtools.UIOutput(text=file)
    display(uio)
    uio.status = "Downloading..."
    
    File_Name = os.path.basename(file)

    response = remote_file.open()
    CHUNK = 16 * 1024
    with open(File_Name, 'wb') as f:
        while True:
            chunk = response.read(CHUNK)
            if not chunk:
                break
            f.write(chunk)
    uio.status = "Downloaded!"
    print(File_Name)
    display(nbtools.UIOutput(files=[File_Name]))
    
genepattern.GPUIBuilder(DownloadJobResultFile, collapse=False,
                    name='Download File From GenePattern Server',
                    description=DownloadJobResultFile_description,
                    parameters=DownloadJobResultFile_parameters)

UIBuilder(collapse=False, description='Download file from a GenePattern module job result.', function_import='…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Download the RDS file.
</div>
<div class="well well-sm">
<b>Note:</b> This takes about 30 seconds on a <i>high speed</i> connnection, please be patient.
</div>

In [15]:
genepattern.GPUIBuilder(DownloadJobResultFile, collapse=False,
                    name='Download File From GenePattern Server',
                    description=DownloadJobResultFile_description,
                    parameters=DownloadJobResultFile_parameters)

UIBuilder(collapse=False, description='Download file from a GenePattern module job result.', function_import='…

## Load data into notebook

In [60]:
%%r_build_ui {"name":"Load dataset with clustering", "parameters": {"RDS_url":{"name":"RDS file","default":"Seurat_preprocessed.clustered.rds.rds",'type':"file"}, "CSV_url":{"name":"CSV_file","type":"file","default":"Seurat_preprocessed.clustered.rds.csv"}, "output_var": {"hide": "True"} }}

load_markers <- function(CSV_url) {
    write("Loading cluster markers into notebook...", stdout())
    markers <- read.csv(CSV_url)
    write("Done!", stdout())
    return(markers)
}
load_it <- function(RDS_url){
    write("Loading clustering results into notebook...", stdout())
    pbmc <- readRDS(file = RDS_url)
    write("Loaded file!", stdout())
    return(pbmc)
}
suppressWarnings(markers <- load_markers(CSV_url))
pbmc <- load_it(RDS_url)

UIBuilder(function_import='nbtools._r_wrappers["5F73CA99F5EEB32D1CF47636D624B986"]', name='Load dataset with c…

In [58]:
%%r_build_ui {"name": "Visualize clusters", "parameters": {"output_var": {"hide": "True"}}}
library(Seurat)
do_dim_plot <- function() {
    plot(DimPlot(pbmc, reduction = "umap"))
    return("")
}
do_dim_plot()

UIBuilder(function_import='nbtools._r_wrappers["8159EC018310FB1FF49E98A5A61016DD"]', name='Visualize clusters'…

In [59]:
%%r_build_ui {"name": "Print top cluster markers", "parameters": {"cluster_number": {}, "output_var": {"hide": "True"}}}
print_top_markers <- function(cluster_number) {
    return(head(markers[markers$cluster==cluster_number,], n = 5))
}
print_top_markers(cluster_number)

UIBuilder(function_import='nbtools._r_wrappers["B14173BB5E052FC7CA9D6C352178E7C4"]', name='Print top cluster m…

In [57]:
%%r_build_ui {"name": "Violin plot of gene expression", "parameters": {"gene": {}, "output_var": {"hide": "True"}}}
do_violin <- function(gene) {
    plot(VlnPlot(pbmc, features = c(gene), slot = "counts", log = TRUE))
    return("")
}
do_violin(gene)

UIBuilder(function_import='nbtools._r_wrappers["538C97A2B8CC11DF57961C4D36D4BD21"]', name='Violin plot of gene…

In [56]:
%%r_build_ui {"name": "Expression of gene across all clusters", "parameters": {"gene": {}, "output_var": {"hide": "True"}}}
do_umap_gene <- function(gene) {
    plot(FeaturePlot(pbmc, features = c(gene)))
    return("")
}
do_umap_gene(gene)

UIBuilder(function_import='nbtools._r_wrappers["D3CC74669A3F0C2F4AC3A95972B63152"]', name='Expression of gene …